In [1]:
!curl -fsSL https://ollama.com/install.sh | sh


>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!ollama serve

In [4]:
from langchain_community.llms import Ollama


In [6]:
llm = Ollama(model="llama2")
llm.invoke("The first man on the moon was ...")

'\nThe first man to walk on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared "That\'s one small step for man, one giant leap for mankind" as he became the first person to walk on the moon.'

In [7]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

'\nCertainly! Here are the categories for each of the expenses you provided:\n\nTaxi Utrecht - Transportation\nMinisterie van Justitie en Veiligheid - Government\nEtos AMSTERDAM NLD - Retail\nBistro Bar Amsterdam - Food and Beverage'

In [8]:
# Read the transactions_2022_2023.csv file
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63


In [9]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)


23

In [10]:
unique_transactions[1:10]


array(['Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

In [11]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 23]

In [12]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)

    return categories_df

In [13]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['', 'Sure! Here are the appropriate categories for each expense:', '', '1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam Nld - Sport', '3. ISS Catering Services De Meern - Food', '4. Taxi Utrecht - Transportation', '5. Etos AMSTERDAM NLD - Grocery', '6. Bistro Bar Amsterdam - Food']


,Transaction vs category,Transaction,Category
0,,,None
1,Sure! Here are the appropriate categories for ...,Sure! Here are the appropriate categories for ...,None
2,,,None
3,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
4,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
5,3. ISS Catering Services De Meern - Food,3. ISS Catering Services De Meern,Food
6,4. Taxi Utrecht - Transportation,4. Taxi Utrecht,Transportation
7,5. Etos AMSTERDAM NLD - Grocery,5. Etos AMSTERDAM NLD,Grocery
8,6. Bistro Bar Amsterdam - Food,6. Bistro Bar Amsterdam,Food


In [14]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['', 'Sure, I can add categories to your expenses. Here is the list of expenses with appropriate categories:', '', '1. Belastingdienst - Taxes', '2. Tesco Breda - Groceries', '3. Monthly Appartment Rent - Housing', '4. Vishandel Sier Amsterdam - Food', '5. Selling Paintings - Arts and Crafts', '6. Spotify Ab By Adyen - Entertainment', '7. Tk Maxx Amsterdam Da - Shopping', '8. Consulting - Professional Services', '9. Aidsfonds - Charity', '10. TLS Bv Inz Ov-Chipkaart - Transportation', '11. Etos Amsterdam - Groceries', '12. Beta Boulders Ams Amsterdam - Entertainment', '13. Salary - Personal Expenses', '14. Bouldermuur Bv Amsterdam - Professional Services', '15. Birtat Restaurant Amsterdam - Food', '16. Freelancing - Personal Expenses', '17. Tikkie - Financial Services', '18. Blogging - Personal Expenses', '19. Taxi Utrecht - Transportation', '20. Apple Services - Technology', '21. Amazon Lux - Shopping', '22. Classpass* Monthly - Fitness', '23. Audible Uk AdblCo/Pymt Gbr - Entertainmen

In [15]:
categories_df_all


,Transaction vs category,Transaction,Category
0,,,None
1,"Sure, I can add categories to your expenses. H...","Sure, I can add categories to your expenses. H...",None
2,,,None
3,1. Belastingdienst - Taxes,1. Belastingdienst,Taxes
4,2. Tesco Breda - Groceries,2. Tesco Breda,Groceries
5,3. Monthly Appartment Rent - Housing,3. Monthly Appartment Rent,Housing
6,4. Vishandel Sier Amsterdam - Food,4. Vishandel Sier Amsterdam,Food
7,5. Selling Paintings - Arts and Crafts,5. Selling Paintings,Arts and Crafts
8,6. Spotify Ab By Adyen - Entertainment,6. Spotify Ab By Adyen,Entertainment
9,7. Tk Maxx Amsterdam Da - Shopping,7. Tk Maxx Amsterdam Da,Shopping


In [16]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)


In [17]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Taxes', 'Groceries', 'Housing', 'Food', 'Arts and Crafts',
       'Entertainment', 'Shopping', 'Professional Services', 'Charity',
       'Transportation', 'Personal Expenses', 'Financial Services',
       'Technology', 'Fitness'], dtype=object)

In [18]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

<ipython-input-18-ca24a9b974c6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
<ipython-input-18-ca24a9b974c6>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
<ipython-input-18-ca24a9b974c6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [19]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

<ipython-input-19-b0c48b6822ef>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
<ipython-input-19-b0c48b6822ef>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')


,Transaction vs category,Transaction,Category
3,1. Belastingdienst - Taxes,Belastingdienst,Taxes
4,2. Tesco Breda - Groceries,Tesco Breda,Groceries
5,3. Monthly Appartment Rent - Housing,Monthly Appartment Rent,Housing
6,4. Vishandel Sier Amsterdam - Food,Vishandel Sier Amsterdam,Food and Drinks
7,5. Selling Paintings - Arts and Crafts,Selling Paintings,Arts and Crafts
8,6. Spotify Ab By Adyen - Entertainment,Spotify Ab By Adyen,Entertainment
9,7. Tk Maxx Amsterdam Da - Shopping,Tk Maxx Amsterdam Da,Shopping
10,8. Consulting - Professional Services,Consulting,Services
11,9. Aidsfonds - Charity,Aidsfonds,Charity
12,10. TLS Bv Inz Ov-Chipkaart - Transportation,TLS Bv Inz Ov-Chipkaart,Transportation


In [20]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,1. Belastingdienst - Taxes,Belastingdienst,Taxes
1,2023-12-30,Tesco Breda,Expense,17.53,2. Tesco Breda - Groceries,Tesco Breda,Groceries
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,3. Monthly Appartment Rent - Housing,Monthly Appartment Rent,Housing
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,4. Vishandel Sier Amsterdam - Food,Vishandel Sier Amsterdam,Food and Drinks
4,2023-12-29,Selling Paintings,Income,13.63,5. Selling Paintings - Arts and Crafts,Selling Paintings,Arts and Crafts
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,6. Spotify Ab By Adyen - Entertainment,Spotify Ab By Adyen,Entertainment
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,7. Tk Maxx Amsterdam Da - Shopping,Tk Maxx Amsterdam Da,Shopping
7,2023-12-22,Consulting,Income,541.57,8. Consulting - Professional Services,Consulting,Services
8,2023-12-22,Aidsfonds,Expense,10.70,9. Aidsfonds - Charity,Aidsfonds,Charity
9,2023-12-20,Consulting,Income,2641.93,8. Consulting - Professional Services,Consulting,Services


In [21]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)

In [22]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

In [23]:
# Read transactions_2022_2023_categorized.csv
df = pd.read_csv('transactions_2022_2023_categorized.csv')
# Add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Month Name'] = pd.to_datetime(df['Date']).dt.strftime("%b")
# Remove "Transaction" and "Transaction vs category" columns
df = df.drop(columns=['Transaction', 'Transaction vs category'])
df

,Date,Name / Description,Expense/Income,Amount (EUR),Category,Year,Month,Month Name
0,2023-12-30,Belastingdienst,Expense,9.96,Taxes,2023,12,Dec
1,2023-12-30,Tesco Breda,Expense,17.53,Groceries,2023,12,Dec
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,Housing,2023,12,Dec
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,Food and Drinks,2023,12,Dec
4,2023-12-29,Selling Paintings,Income,13.63,Arts and Crafts,2023,12,Dec
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,Entertainment,2023,12,Dec
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,Shopping,2023,12,Dec
7,2023-12-22,Consulting,Income,541.57,Services,2023,12,Dec
8,2023-12-22,Aidsfonds,Expense,10.70,Charity,2023,12,Dec
9,2023-12-20,Consulting,Income,2641.93,Services,2023,12,Dec


In [24]:
# For Income rows, assign Name / Description to Category
df['Category'] = np.where(df['Expense/Income'] == 'Income', df['Name / Description'], df['Category'])

In [25]:
def make_pie_chart(df, year, label):
    # Filter the dataset for expense transactions
    sub_df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]

    color_scale = px.colors.qualitative.Set2

    pie_fig = px.pie(sub_df, values='Amount (EUR)', names='Category', color_discrete_sequence = color_scale)
    pie_fig.update_traces(textposition='inside', direction ='clockwise', hole=0.3, textinfo="label+percent")

    total_expense = df[(df['Expense/Income'] == 'Expense') & (df['Year'] == year)]['Amount (EUR)'].sum()
    total_income = df[(df['Expense/Income'] == 'Income') & (df['Year'] == year)]['Amount (EUR)'].sum()

    if label == 'Expense':
        total_text = "€ " + str(round(total_expense))

        # Saving rate:
        saving_rate = round((total_income - total_expense)/total_income*100)
        saving_rate_text = ": Saving rate " + str(saving_rate) + "%"
    else:
        saving_rate_text = ""
        total_text = "€ " + str(round(total_income))

    pie_fig.update_layout(uniformtext_minsize=10,
                        uniformtext_mode='hide',
                        title=dict(text=label+" Breakdown " + str(year) + saving_rate_text),
                        # Add annotations in the center of the donut.
                        annotations=[
                            dict(
                                text=total_text,
                                # Square unit grid starting at bottom left of page
                                x=0.5, y=0.5, font_size=12,
                                # Hide the arrow that points to the [x,y] coordinate
                                showarrow=False
                            )
                        ]
                    )
    return pie_fig

In [26]:
income_pie_fig_2022 = make_pie_chart(df, 2022, 'Income')
income_pie_fig_2022

In [27]:
def make_monthly_bar_chart(df, year, label):
    df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month', 'Month Name'])['Amount (EUR)'].sum()
                        .to_frame()
                        .reset_index()
                        .sort_values(by='Month')
                        .reset_index(drop=True))
    if label == "Income":
        color_scale = px.colors.sequential.YlGn
    if label == "Expense":
        color_scale = px.colors.sequential.OrRd

    bar_fig = px.bar(total_by_month, x='Month Name', y='Amount (EUR)', text_auto='.2s', title=label+" per month", color='Amount (EUR)', color_continuous_scale=color_scale)
    # bar_fig.update_traces(marker_color='lightslategrey')

    return bar_fig

In [28]:
income_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Income')
income_monthly_2022

In [29]:
# Pie charts
income_pie_fig_2022 = make_pie_chart(df, 2022, 'Income')
expense_pie_fig_2022 = make_pie_chart(df, 2022, 'Expense')
income_pie_fig_2023 = make_pie_chart(df, 2023, 'Income')
expense_pie_fig_2023 = make_pie_chart(df, 2023, 'Expense')

# Bar charts
income_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Income')
expense_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Expense')
income_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Income')
expense_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Expense')

# Create tabs
tabs = pn.Tabs(
                        ('2022', pn.Column(pn.Row(income_pie_fig_2022, expense_pie_fig_2022),
                                                pn.Row(income_monthly_2022, expense_monthly_2022))),
                        ('2023', pn.Column(pn.Row(income_pie_fig_2023, expense_pie_fig_2023),
                                                pn.Row(income_monthly_2023, expense_monthly_2023))
                        )
                )
tabs.show()

Launching server at http://localhost:34277


In [32]:
# Dashboard template
template = pn.template.FastListTemplate(
    title='Personal Finance Dashboard',
    sidebar=[pn.pane.Markdown("# Income Expense analysis"),
             pn.pane.Markdown("Overview of income and expense based on my bank transactions. Categories are obtained using local LLMs."),
             pn.pane.PNG("picture.png", sizing_mode="scale_both")
             ],
    main=[pn.Row(pn.Column(pn.Row(tabs)
                           )
                ),
                ],
    # accent_base_color="#88d8b0",
    header_background="#c0b9dd",
)

template.show()

Launching server at http://localhost:32795
